In [7]:
import spynnaker8 as p
from pyNN.random import NumpyRNG, RandomDistribution
from pyNN.parameters import Sequence
from pyNN.utility.plotting import Figure, Panel
import matplotlib.pyplot as plt

import time
import numpy as np
import math
import cv2
import pdb


import sys
sys.path.insert(1, '../../miscelaneous')
from spinnorse_tools import fetch_files, plot_in_v_out, plot_spikes, plot_voltages
from stimulator import produce_data

In [2]:
duration = 3
l_px = 32
w_px = int(l_px*3/4)
vx =  l_px/80
vy = -w_px/160
r_ball = int(3+w_px/48)
mat, coor = produce_data(l_px, w_px, r_ball, vx, vy, duration)
# print(mat[3,4,:])

In [3]:
dt = 1           # (ms) simulation timestep

cell_params = {'tau_m': 20.0,
               'tau_syn_E': 5.0,
               'tau_syn_I': 5.0,
               'v_rest': -65.0,
               'v_reset': -65.0,
               'v_thresh': -50.0,
               'tau_refrac': 0.0, # 0.1 originally
               'cm': 1,
               'i_offset': 0.0
               }

w = 1.0
w_fovea = 0.5

In [16]:
nb_steps = duration*1000


#SpiNNaker Setup
delay = dt
node_id = p.setup(timestep=dt, min_delay=delay, max_delay=delay)     
p.set_number_of_neurons_per_core(p.IF_curr_exp, 100) #  100 neurons per core


celltype = p.IF_curr_exp

cells_l0 = []  
cells_l1 = []   
con_l0l1 = []  
con_move = []
w_x = np.zeros((2,l_px))
w_y = np.zeros((2,w_px))

idx = -1

m_labels = ["go_right", "go_left", "go_up", "go_down"]
colors = ["g", "r", "g", "r"]

move_neurons = []
for i in range(len(m_labels)):
    move_neurons.append(p.Population(1, celltype(**cell_params), label=m_labels[i]) )
    move_neurons[i].record(["v","spikes"])

i_indexes = []
for y in range(w_px):
        
    for x in range(l_px):
        
        idx += 1
        i_spikes = mat[y, x, :]
        i_indexes.append(np.where(i_spikes > 0))

cells_l0 = p.Population(w_px*l_px, p.SpikeSourceArray(spike_times=(i_indexes)))        
        
#         cur_label = "N_{:d}_{:d}".format(x, y)
        
#         # Populations        
#         cells_l0.append(p.Population(1,spike_trains))   

#         # Connectivity
#         cell_conn = p.AllToAllConnector()
#         cur_label = "i2l_{:d}_{:d}".format(x, y)         
        
#         # Move right (when stimulus on the left 'hemisphere')    
#         if x < l_px/2:
#             w_motor_x = w_fovea*(abs(x-l_px/2)/l_px )
#             con_move.append({ 'l1lm': p.Projection(cells_l0[idx], move_neurons[0], cell_conn,
#                                             receptor_type='excitatory',
#                                             synapse_type=p.StaticSynapse(weight=w_motor_x, delay=delay))})
    
#         # Move Left (when stimulus on the right 'hemisphere')
#         if x > l_px/2:
#             w_motor_x = w_fovea*(abs(x-l_px/2)/l_px )
#             con_move.append({ 'l1lm': p.Projection(cells_l0[idx], move_neurons[1], cell_conn,
#                                             receptor_type='excitatory',
#                                             synapse_type=p.StaticSynapse(weight=w_motor_x, delay=delay))})
        
#         # Move up (when stimulus on the bottom 'hemisphere')     
#         if y < w_px/2: # higher pixel --> bottom of image
#             w_motor_y = w_fovea*(abs(y-w_px/2)/w_px )
#             con_move.append({ 'l1lm': p.Projection(cells_l0[idx], move_neurons[2], cell_conn,
#                                             receptor_type='excitatory',
#                                             synapse_type=p.StaticSynapse(weight=w_motor_y, delay=delay))})
        
#         # Move down (when stimulus on the top 'hemisphere') 
#         if y > w_px/2: # lower pixel --> top of image
#             w_motor_y = w_fovea*(abs(y-w_px/2)/w_px )
#             con_move.append({ 'l1lm': p.Projection(cells_l0[idx], move_neurons[3], cell_conn,
#                                             receptor_type='excitatory',
#                                             synapse_type=p.StaticSynapse(weight=w_motor_y, delay=delay))})
            
        
# Run simulation 
p.run(nb_steps)



        


2022-03-18 16:03:43 WARNING: Calling setup a second time causes the previous simulator to be stopped and cleared.
2022-03-18 16:03:43 INFO: Caching data for segment -1
2022-03-18 16:03:43 WARNING: The simulation has not yet run, therefore spikes cannot be retrieved, hence the list will be empty
2022-03-18 16:03:43 WARNING: The simulation has not yet run, therefore v cannot be retrieved, hence the list will be empty
2022-03-18 16:03:43 INFO: Caching data for segment -1
2022-03-18 16:03:43 WARNING: The simulation has not yet run, therefore spikes cannot be retrieved, hence the list will be empty
2022-03-18 16:03:43 WARNING: The simulation has not yet run, therefore v cannot be retrieved, hence the list will be empty
2022-03-18 16:03:43 INFO: Caching data for segment -1
2022-03-18 16:03:43 WARNING: The simulation has not yet run, therefore spikes cannot be retrieved, hence the list will be empty
2022-03-18 16:03:43 WARNING: The simulation has not yet run, therefore v cannot be retrieved, 

['/home/juan/spinstuff/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/juan/spinstuff/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg', '/home/juan/.spynnaker.cfg']


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 0 and the array at index 48 has size 4

In [ ]:

m_indexes = []
m_v_arrays = []
for i in range(len(m_labels)):    
    spikes = move_neurons[i].get_data("spikes")
    voltage = move_neurons[i].get_data("v")
    m_indexes.append(np.asarray(spikes.segments[0].spiketrains[0]))
    m_v_arrays.append(np.array(voltage.segments[0].filter(name="v")[0]).reshape(-1))


In [ ]:

p.end()

In [ ]:

xlim = nb_steps       
for i in range(4):
    plot_spikes(m_indexes[i:i+1], m_labels[i], xlim)
# for i in range(4):
#     plot_voltages(m_v_arrays[i:i+1], m_labels[i], xlim)

In [ ]:

win_size = 20

fig, axs = plt.subplots(2, figsize=(15,6))
fig.tight_layout(pad=5.0)
fig.suptitle("Trajectories")  


# print("{:.1f} %".format(np.round(t/(duration*1000),3)*100))
max_y = 0
for j in range(4):
    neuron_nb = j
    count = np.zeros(nb_steps-win_size)
    for i in range(win_size, nb_steps, 1):
        
        a = np.squeeze(np.array(np.where((m_indexes[neuron_nb]>=(i-win_size)) & (m_indexes[neuron_nb]<=(i)))), axis=0)
        count[i-win_size] = len(a)
        if count[i-win_size] > max_y:
            max_y = count[i-win_size]

    subplot_idx = int(j/2)
    axs[subplot_idx].plot(count, label=m_labels[j], color=colors[j])
    axs[subplot_idx].legend()

    
axs[0].set_ylim([0, int(max_y*1.2)])
axs[1].set_ylim([0, int(max_y*1.2)])
plt.legend()
# plt.xlim([0, 20])

In [ ]:
fig, axs = plt.subplots(1, figsize=(15,6))
fig.tight_layout(pad=5.0)
fig.suptitle("Trajectories")  
axs.plot(coor[0,:], label="x", color="b")
axs.plot(coor[1,:], label="y", color="y")
axs.plot(np.round(coor[0,:],0), label="x", color="b")
axs.plot(np.round(coor[1,:],0), label="y", color="y")
axs.set_ylim([0, 20])
plt.legend() 
    

In [18]:
def generate_spike_times(i_range):
    return [Sequence(np.add.accumulate(np.random.exponential(10.0, size=10)))
            for i in i_range]

a = generate_spike_times([1,2])
print(a)

[Sequence([ 3.9269001   8.53628472 21.66986342 28.09319514 29.23213537 32.85930484
 36.3052331  45.35326238 52.57282478 59.06147085]), Sequence([ 5.49313381  5.64379471 19.05099009 39.51459371 39.67042173 45.20041572
 49.81070531 60.90835947 65.48834456 66.168604  ])]


In [17]:
np.add.accumulate(np.random.exponential(10.0, size=10))

array([ 3.71797867,  5.19567339, 21.8856394 , 34.26283856, 38.44386064,
       43.61033164, 51.48158034, 59.11617165, 69.80679665, 69.89956555])